# Calculate global overturning circulation

In [1]:
import cosima_cookbook as cc
from dask.distributed import Client

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmocean as cm
import matplotlib.gridspec as gridspec
%matplotlib inline

# Stop annoying warnings coming out of xarray.
import warnings
warnings.filterwarnings('ignore')

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/42723/status,
Dashboard: /proxy/42723/status,Workers: 7
Total threads: 28,Total memory: 125.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42169,Workers: 7
Dashboard: /proxy/42723/status,Total threads: 28
Started: Just now,Total memory: 125.20 GiB
Comm: tcp://127.0.0.1:37097,Total threads: 4
Dashboard: /proxy/42699/status,Memory: 17.89 GiB
Nanny: tcp://127.0.0.1:43471,


In [3]:
# CM2 database
session_CM2 = cc.database.create_session('/g/data/p73/archive/non-CMIP/ACCESS-CM2/CM2.db')
expt_CM2_025 = 'cj877'
expt_CM2_1   = 'bz687'

In [4]:
# OM2 database
session_OM2  = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
expt_OM2_025 = '025deg_jra55_ryf9091_gadi'
expt_OM2_1   = '1deg_jra55_ryf9091_gadi'

In [5]:
path_to_data = '/g/data/x77/wgh581/Post_Process/ACCESS_CM2_025/'

If you get a 'read-only database' warning, it is because the database has not finished building. 

In [6]:
# Plotting info
ft_size = 16
plt.rcParams.update({'font.size': ft_size})
fig_path = '/g/data/x77/wgh581/Figures/Figures_ACCESS_CM2/'

### Calculate overturning

In [7]:
def compute_psi_rho(expt, session, frequency='1 monthly', nbound=None, start_time=None, end_time=None):
    rho = 1025 # mean density of sea-water in kg/m^3
    
    varlist = cc.querying.get_variables(session, expt)
    if varlist['name'].str.contains('ty_trans_rho_gm').any():
        GM = True
        print('GM is True')
        psiGM = cc.querying.getvar(expt, 'ty_trans_rho_gm', session, frequency=frequency, n=nbound, start_time=start_time, end_time=end_time)
        psiGM = psiGM.sum('grid_xt_ocean')
        psiGM = psiGM / (1e6*rho)
    else:
        GM = False
        print('GM is False')
        
    psi = cc.querying.getvar(expt, 'ty_trans_rho', session, frequency=frequency, n=nbound, start_time=start_time, end_time=end_time)      
    psi = psi / (1e6*rho) # converts kg/s to Sv
    psi = psi.sum('grid_xt_ocean').cumsum('potrho').mean(dim='time').load() 
    if GM:
        psi = psi + psiGM.mean('time')
        
    return psi.compute()

CM2-025

In [ ]:
year = np.arange(1, 500, 1)

for ii in year:
    
    if ii > 341:
        if len(str(int(ii))) == 1:
            year_tmp = '000' +str(ii)
        elif len(str(int(ii))) == 2:
            year_tmp = '00' + str(ii)
        elif len(str(int(ii))) == 3:
            year_tmp = '0' + str(ii)

        start_time = year_tmp + '-01-01'
        end_time   = year_tmp + '-12-31'
        data_name   = 'Overturning_' + year_tmp + '.nc'

        psi_tmp = compute_psi_rho(expt_CM2_025, session_CM2, start_time=start_time, end_time=end_time)
        psi_tmp.to_netcdf(path_to_data + 'overturning_tmp/' + data_name)
        print('Finished year ', year_tmp)

CM2-1

In [ ]:
year = np.arange(400, 500, 1)

for ii in year:
    
    year_tmp = '0' + str(ii)

    start_time = year_tmp + '-01-01'
    end_time   = year_tmp + '-12-31'
    data_name  = 'Overturning_CM2_1_' + year_tmp + '.nc'

    psi_tmp = compute_psi_rho(expt_CM2_1, session_CM2, start_time=start_time, end_time=end_time)
    psi_tmp.to_netcdf(path_to_data + '/Comparison_ACCESS_CM2_1/overturning_tmp/' + data_name)
    print('Finished year ', year_tmp)

OM2-025

In [ ]:
year = np.arange(2300, 2399, 1)

#for ii in year:
for ii in [2399]:    
    year_tmp = str(ii)

    start_time = year_tmp + '-01-01'
    end_time   = year_tmp + '-12-31'
    data_name  = 'Overturning_OM2_025_' + year_tmp + '.nc'

    psi_tmp = compute_psi_rho(expt_OM2_025, session_OM2, frequency='1 monthly', start_time=start_time, end_time=end_time)
    psi_tmp.to_netcdf(path_to_data + '/Comparison_ACCESS_OM2_025/overturning_tmp/' + data_name)
    print('Finished year ', year_tmp)

OM2-1

In [8]:
year = np.arange(2500, 2599, 1)

#for ii in year:
for ii in [2599]:
    
    year_tmp = str(ii)

    start_time = year_tmp + '-01-01'
    end_time   = year_tmp + '-12-31'
    data_name  = 'Overturning_OM2_1_' + year_tmp + '.nc'

    psi_tmp = compute_psi_rho(expt_OM2_1, session_OM2, frequency='1 monthly', start_time=start_time, end_time=end_time)
    psi_tmp.to_netcdf(path_to_data + '/Comparison_ACCESS_OM2_1/overturning_tmp/' + data_name)
    print('Finished year ', year_tmp)

GM is True


/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datet

Finished year  2599


### Load overturning data (individual files), concatenate and save output as one file

In [16]:
import glob
import xesmf
import os
import pandas as pd
import calendar
from tqdm import tqdm_notebook

In [21]:
def load_individual_files(file_name='overturning_tmp/*.nc'):
    dataarrays = []
    files = glob.glob(os.path.join(path_to_data, file_name))
    for f in tqdm_notebook(files, leave=False, desc='opening files'):
        dataarrays.append(xr.open_dataarray(f, decode_times=False))
    dataarray = xr.concat(dataarrays, dim='time', coords='all')
    if 'time' in dataarray.coords:
        time_units = dataarray.time.units
        decoded_time = xr.conventions.times.decode_cf_datetime(dataarray.time, time_units)
        dataarray.coords['time'] = ('time', decoded_time,
                                    {'long_name': 'time', 'decoded_using': time_units })
    return dataarray.load()

In [19]:
psi_CM2_025 = load_individual_files()

In [22]:
psi_CM2_1 = load_individual_files(file_name = 'Comparison_ACCESS_CM2_1/overturning_tmp/*.nc')

opening files:   0%|          | 0/100 [00:00<?, ?it/s]

In [25]:
psi_OM2_1 = load_individual_files(file_name = 'Comparison_ACCESS_OM2_1/overturning_tmp/*.nc')

opening files:   0%|          | 0/100 [00:00<?, ?it/s]

In [26]:
psi_OM2_025 = load_individual_files(file_name = 'Comparison_ACCESS_OM2_025/overturning_tmp/*.nc')

opening files:   0%|          | 0/100 [00:00<?, ?it/s]

In [27]:
psi_CM2_025.to_netcdf(path_to_data + 'psi_global_0_499.nc')

In [28]:
psi_CM2_1.to_netcdf(path_to_data + 'Comparison_ACCESS_CM2_1/psi_global_CM2_1_400_499.nc')

In [29]:
psi_OM2_1.to_netcdf(path_to_data + 'Comparison_ACCESS_OM2_1/psi_global_OM2_1_2500_2599.nc')

In [30]:
psi_OM2_025.to_netcdf(path_to_data + 'Comparison_ACCESS_OM2_025/psi_global_OM2_025_2300_2399.nc')

### Test if files work fine and then delete the infividual files/folders

### Abyssal overturning at 40S

In [7]:
def get_AABW_40S(expt, session, start_time, end_time):
    
    psi = cc.querying.getvar(expt, 'ty_trans_rho', session, chunks={'potrho': None}, start_time=start_time, end_time=end_time)
    psi = psi.sel(time=slice(start_time, end_time))
    psi = psi.sum('grid_xt_ocean').sel(method='Nearest',grid_yu_ocean=-40)*1.0e-9
    psi_sum = psi.cumsum('potrho') - psi.sum('potrho')
    
    ## If GM overturning is output, load that too
    try:
        psiGM = cc.querying.getvar(expt, 'ty_trans_rho_gm', session, chunks={'potrho': None}, start_time=start_time, end_time=end_time)
        psiGM = psiGM.sel(time=slice(start_time, end_time))
        psiGM = psiGM.sum('grid_xt_ocean').sel(method='Nearest',grid_yu_ocean=-40)*1.0e-9
        psi_sum = psi_sum + psiGM
    except:
        print('GM term not found')
        
    psi_aabw = -psi_sum.sel(potrho=slice(1036,None)).min('potrho').resample(time='Y').mean('time')

    return psi_aabw.compute()

CM2-025

In [29]:
%%time
year = np.arange(400, 499, 1)

for ii in range(len(year)):
    start_time = '0' + str(year[ii]) + '-01-01'
    end_time = '0' + str(year[ii]) + '-12-31'
    psi_aabw = get_AABW_40S(expt_CM2_025, session_CM2, start_time, end_time)
    
    if ii == 0:
        psi_aabw_all = psi_aabw
    else:
        psi_aabw_all = xr.concat([psi_aabw_all, psi_aabw], dim='time')
    
    psi_aabw_all.to_netcdf(path_to_data + 'psi_AMOC_40S_annual_yr_400_499.nc')

CPU times: user 3min 35s, sys: 1min 1s, total: 4min 36s
Wall time: 32min 22s


In [33]:
psi_aabw_all.to_netcdf(path_to_data + 'psi_AMOC_40S_annual_yr_400_499.nc')

In [ ]:
test = xr.open_dataarray(path_to_data + 'psi_AMOC_40S_annual_yr_400_499.nc')

In [10]:
test.mean()

<xarray.DataArray ()>
array(14.592813, dtype=float32)
Coordinates:
    grid_yu_ocean  float64 -40.03

CM2-1

In [ ]:
%%time
year = np.arange(400, 499, 1)

for ii in range(len(year)):
    start_time = '0' + str(year[ii]) + '-01-01'
    end_time = '0' + str(year[ii]) + '-12-31'
    psi_aabw = get_AABW_40S(expt_CM2_1, session_CM2, start_time, end_time)
    
    if ii == 0:
        psi_aabw_all = psi_aabw
    else:
        psi_aabw_all = xr.concat([psi_aabw_all, psi_aabw], dim='time')
    
    psi_aabw_all.to_netcdf(path_to_data + '/Comparison_ACCESS_CM2_1/psi_AMOC_40S_annual_CM2_1_yr_400_499.nc')

OM2-025

In [24]:
# Running the 0M2-025 did take a really long time and I had to break it up into individual time bits. (Also OM2-025 does NOT have the gm component saved, although it is turned on.)

In [ ]:
%%time
year = np.arange(2300, 2400, 1)

for ii in range(len(year)):
    start_time = str(year[ii]) + '-01-01'
    end_time = str(year[ii]) + '-12-31'
    psi_aabw = get_AABW_40S(expt_OM2_025, session_OM2, start_time, end_time)
    
    if ii == 0:
        psi_aabw_all = psi_aabw
    else:
        psi_aabw_all = xr.concat([psi_aabw_all, psi_aabw], dim='time')
    
    psi_aabw_all.to_netcdf(path_to_data + '/Comparison_ACCESS_OM2_025/psi_AMOC_40S_annual_OM2_025_yr_2300_2399.nc')

OM2-1

In [ ]:
%%time
year = np.arange(2500, 2599, 1)

for ii in range(len(year)):
    start_time = str(year[ii]) + '-01-01'
    end_time = str(year[ii]) + '-12-31'
    
    psi_aabw = get_AABW_40S(expt_OM2_1, session_OM2, start_time, end_time)
    
    if ii == 0:
        psi_aabw_all = psi_aabw
    else:
        psi_aabw_all = xr.concat([psi_aabw_all, psi_aabw], dim='time')
    
    psi_aabw_all.to_netcdf(path_to_data + '/Comparison_ACCESS_OM2_1/psi_AMOC_40S_annual_OM2_1_yr_2500_2599.nc')

### Overturning per basin

100 year average

Function to create mask for Atlantic and Indopacific (both include the Southern Ocean). For details see
https://github.com/COSIMA/cosima-recipes/blob/master/DocumentedExamples/Atlantic_IndoPacific_Basin_Overturning_Circulation.ipynb

In [7]:
def get_mask(expt, session):
    
    # Load bathymetry
    ht = cc.querying.getvar(expt, 'ht', session, n=-1)
    
    # Mask array (1 ocean, 0 land)
    land_mask = ~ht.isnull()
    
    ## create masks out of the above chunks
    south_map = (land_mask.where(land_mask.yt_ocean < -34)).fillna(0) 
    indo_map1 = (land_mask.where(land_mask.yt_ocean < 9).where(land_mask.yt_ocean > -34).where(land_mask.xt_ocean >-280).where(land_mask.xt_ocean<-65)).fillna(0) 
    indo_map2 = (land_mask.where(land_mask.yt_ocean < 15).where(land_mask.yt_ocean > 9).where(land_mask.xt_ocean >-280).where(land_mask.xt_ocean<-83.7)).fillna(0) 
    indo_map3 = (land_mask.where(land_mask.yt_ocean < 17).where(land_mask.yt_ocean > 15).where(land_mask.xt_ocean >-280).where(land_mask.xt_ocean<-93.3)).fillna(0) 
    indo_map4 = (land_mask.where(land_mask.yt_ocean < 85).where(land_mask.yt_ocean > 17).where(land_mask.xt_ocean >-280).where(land_mask.xt_ocean<-99)).fillna(0) 
    indo_map5 = (land_mask.where(land_mask.yt_ocean < 30.5).where(land_mask.yt_ocean > -34).where(land_mask.xt_ocean >25).where(land_mask.xt_ocean<80)).fillna(0) 
    indo_sector_map = indo_map1 + indo_map2 + indo_map3 + indo_map4 + indo_map5 + south_map
    indo_sector_mask = indo_sector_map.where(indo_sector_map>0)
    atlantic_sector_map = (indo_sector_mask * 0).fillna(1) * land_mask
    atlantic_sector_map = atlantic_sector_map + south_map
    atlantic_sector_mask = atlantic_sector_map.where(atlantic_sector_map>0)

    # Mask above is on (yt_ocean, xt_ocean) but we need (yu_ocean, xt_ocean)
    psi = cc.querying.getvar(expt, 'ty_trans_rho', session, n=1) ## needs to be the same coordinates as what you want to mask
    atlantic_sector_mask.coords['xt_ocean'] = psi.grid_xt_ocean.values
    atlantic_sector_mask.coords['yt_ocean'] = psi.grid_yu_ocean.values
    atlantic_sector_mask = atlantic_sector_mask.rename({'xt_ocean':'grid_xt_ocean','yt_ocean':'grid_yu_ocean'})
    indo_sector_mask.coords['xt_ocean'] = psi.grid_xt_ocean.values
    indo_sector_mask.coords['yt_ocean'] = psi.grid_yu_ocean.values
    indo_sector_mask = indo_sector_mask.rename({'xt_ocean':'grid_xt_ocean','yt_ocean':'grid_yu_ocean'})
    
    return atlantic_sector_mask, indo_sector_mask

Function to calculate overturning for specific basin mask

In [8]:
def compute_basin_psi_rho(expt, session, basin_mask, frequency='1 monthly', nbound=None, start_time=None, end_time=None):
    rho = 1025 # mean density of sea-water in kg/m^3
    
    varlist = cc.querying.get_variables(session, expt)
    if varlist['name'].str.contains('ty_trans_rho_gm').any():
        GM = True
        print('GM is True')
        psiGM = cc.querying.getvar(expt, 'ty_trans_rho_gm', session, frequency=frequency, n=nbound, start_time=start_time, end_time=end_time)
        psiGM = psiGM.sum('grid_xt_ocean')
        psiGM = psiGM / (1e6*rho)
    else:
        GM = False
        print('GM is False')
        
    psi = cc.querying.getvar(expt, 'ty_trans_rho', session, frequency=frequency, n=nbound, start_time=start_time, end_time=end_time)      
    psi = psi / (1e6*rho) # converts kg/s to Sv
    psi = (psi * basin_mask).sum('grid_xt_ocean').cumsum('potrho').mean(dim='time').load() 
    if GM:
        psi = psi + psiGM.mean('time')
        
    return psi.compute()

Now make the masks

In [26]:
%%time
atlantic_sector_mask_CM2_025, indo_sector_mask_CM2_025 = get_mask(expt_CM2_025, session_CM2)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, cal

CPU times: user 9.85 s, sys: 1.47 s, total: 11.3 s
Wall time: 15.6 s


In [29]:
%%time
atlantic_sector_mask_CM2_1, indo_sector_mask_CM2_1 = get_mask(expt_CM2_1, session_CM2)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, cal

CPU times: user 2.63 s, sys: 343 ms, total: 2.97 s
Wall time: 5.21 s


In [9]:
%%time
atlantic_sector_mask_OM2_025, indo_sector_mask_OM2_025 = get_mask(expt_OM2_025, session_OM2)

CPU times: user 6.23 s, sys: 1.71 s, total: 7.94 s
Wall time: 30.5 s


In [17]:
%%time
atlantic_sector_mask_OM2_1, indo_sector_mask_OM2_1 = get_mask(expt_OM2_1, session_OM2)

CPU times: user 4.49 s, sys: 630 ms, total: 5.12 s
Wall time: 22.6 s


Calculate overturning for each mask and experiment

Atlantic

In [ ]:
%%time
year = np.arange(400, 500, 1)

for ii in range(len(year)):
    start_time = '0' + str(year[ii]) + '-01-01'
    end_time = '0' + str(year[ii]) + '-12-31'
    
    atlantic_tmp = compute_basin_psi_rho(expt_CM2_025, session_CM2, atlantic_sector_mask_CM2_025, start_time=start_time, end_time=end_time)
    
    if ii == 0:
        atlantic = atlantic_tmp
    else:
        atlantic = xr.concat([atlantic, atlantic_tmp], dim='time')
    
    atlantic.to_netcdf(path_to_data + '/atlantic_psi_400_499.nc')

In [ ]:
%%time
year = np.arange(400, 500, 1)

for ii in range(len(year)):
    start_time = '0' + str(year[ii]) + '-01-01'
    end_time = '0' + str(year[ii]) + '-12-31'
    
    atlantic_tmp = compute_basin_psi_rho(expt_CM2_1, session_CM2, atlantic_sector_mask_CM2_1, start_time=start_time, end_time=end_time)
    
    if ii == 0:
        atlantic = atlantic_tmp
    else:
        atlantic = xr.concat([atlantic, atlantic_tmp], dim='time')
    
    atlantic.to_netcdf(path_to_data + '/Comparison_ACCESS_CM2_1/atlantic_psi_400_499_CM2_1.nc')

In [ ]:
%%time
year = np.arange(2300, 2400, 1)

for ii in range(len(year)):
    start_time = str(year[ii]) + '-01-01'
    end_time = str(year[ii]) + '-12-31'
    
    atlantic_tmp = compute_basin_psi_rho(expt_OM2_025, session_OM2, atlantic_sector_mask_OM2_025, start_time=start_time, end_time=end_time)
    
    if ii == 0:
        atlantic = atlantic_tmp
    else:
        atlantic = xr.concat([atlantic, atlantic_tmp], dim='time')
    
    atlantic.to_netcdf(path_to_data + '/Comparison_ACCESS_OM2_025/atlantic_psi_2300_2399_OM2_025.nc')

In [ ]:
%%time
year = np.arange(2500, 2600, 1)

for ii in range(len(year)):
    start_time = str(year[ii]) + '-01-01'
    end_time = str(year[ii]) + '-12-31'
    
    atlantic_tmp = compute_basin_psi_rho(expt_OM2_1, session_OM2, atlantic_sector_mask_OM2_1, start_time=start_time, end_time=end_time)
    
    if ii == 0:
        atlantic = atlantic_tmp
    else:
        atlantic = xr.concat([atlantic, atlantic_tmp], dim='time')
    
    atlantic.to_netcdf(path_to_data + '/Comparison_ACCESS_OM2_1/atlantic_psi_2500_2599_OM2_1.nc')

Pacific

In [ ]:
%%time
year = np.arange(400, 500, 1)

for ii in range(len(year)):
    start_time = '0' + str(year[ii]) + '-01-01'
    end_time = '0' + str(year[ii]) + '-12-31'
    
    indopacific_tmp = compute_basin_psi_rho(expt_CM2_025, session_CM2, indo_sector_mask_CM2_025, start_time=start_time, end_time=end_time)
    
    if ii == 0:
        indopacific = indopacific_tmp
    else:
        indopacific = xr.concat([indopacific, indopacific_tmp], dim='time')
    
    indopacific.to_netcdf(path_to_data + '/indopacific_psi_400_499.nc')

In [ ]:
%%time
year = np.arange(400, 500, 1)

for ii in range(len(year)):
    start_time = '0' + str(year[ii]) + '-01-01'
    end_time = '0' + str(year[ii]) + '-12-31'
    
    indopacific_tmp = compute_basin_psi_rho(expt_CM2_1, session_CM2, indo_sector_mask_CM2_1, start_time=start_time, end_time=end_time)
    
    if ii == 0:
        indopacific = indopacific_tmp
    else:
        indopacific = xr.concat([indopacific, indopacific_tmp], dim='time')
    
    indopacific.to_netcdf(path_to_data + '/Comparison_ACCESS_CM2_1/indopacific_psi_400_499_CM2_1.nc')

In [ ]:
%%time
year = np.arange(2300, 2400, 1)

for ii in range(len(year)):
    start_time = str(year[ii]) + '-01-01'
    end_time = str(year[ii]) + '-12-31'
    
    indopacific_tmp = compute_basin_psi_rho(expt_OM2_025, session_OM2, indo_sector_mask_OM2_025, start_time=start_time, end_time=end_time)
    
    if ii == 0:
        indopacific = indopacific_tmp
    else:
        indopacific = xr.concat([indopacific, indopacific_tmp], dim='time')
    
    indopacific.to_netcdf(path_to_data + '/Comparison_ACCESS_OM2_025/indopacific_psi_2300_2399_OM2_025.nc')

In [18]:
%%time
year = np.arange(2500, 2600, 1)

for ii in range(len(year)):
    start_time = str(year[ii]) + '-01-01'
    end_time = str(year[ii]) + '-12-31'
    
    indopacific_tmp = compute_basin_psi_rho(expt_OM2_1, session_OM2, indo_sector_mask_OM2_1, start_time=start_time, end_time=end_time)
    
    if ii == 0:
        indopacific = indopacific_tmp
    else:
        indopacific = xr.concat([indopacific, indopacific_tmp], dim='time')
    
    indopacific.to_netcdf(path_to_data + '/Comparison_ACCESS_OM2_1/indopacific_psi_2500_2599_OM2_1.nc')

GM is True


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datet

GM is True


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datet

GM is True


...

GM is True


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datet

GM is True


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/xarray/coding/times.py:710: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datet

CPU times: user 1h 2min 16s, sys: 4min 21s, total: 1h 6min 38s
Wall time: 1h 9min 37s
